# py-装饰器

In [6]:
# 函数计时装饰器装饰器的创建
import time
from functools import wraps

def timer_counter(func):
    '''
    用装饰器实现函数计时
    :param function: 需要计时的函数
    :return: None
    '''
    @wraps(func)
    def function_timer(*args, **kwargs):
        print('[Function: {name} start...]'.format(name = func.__name__))
        t0 = time.time()
        result = func(*args, **kwargs)
        t1 = time.time()
        print('[Function: {name} finished, spent time: {time:.2f}s]'.format(name = func.__name__, time = t1 - t0))
        return result
    return function_timer

@timer_counter
def test_func():
    print("count start ...")
    for i in range(3):
        time.sleep(1)
    print('count end.')


test_func()

[Function: test_func start...]
count start ...
count end.
[Function: test_func finished, spent time: 3.00s]
